# Project 5
# Car price prediction model

## 0. Подготовка

Загружаем необходимые библиотеки, настраиваем отображение, инициализируем глобальные переменные

In [1]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
RANDOM_STATE = 73

## 1. Анализ данных, которые необходимо получить

Прежде, чем начать мучить сайт auto.ru запросами, необходимо понять, какие данные и в каком виде необходимо получить.
Для этого загрузим тестовый датасет (по которому надо будет делать сабмишн) и немного поработаем с ним.

In [2]:
df_test = pd.read_csv('test.csv')

Сразу заменим имена столбцов на PEP-friendly

In [3]:
df_test.columns = ['body_type', 'brand', 'car_url', 'color', 'complectation',
              'description', 'engine_vol', 'engine_power', 'equipment',
              'fuel_type', 'image', 'mileage', 'model_date', 'model_info', 'model_name',
              'name', 'doors_num', 'parsing_unixtime', 'price_currency',
              'production_date', 'sell_id', 'super_gen', 'configuration',
              'transmission', 'vendor', 'num_owners', 'ownership', 'docs',
              'drive', 'steering', 'state', 'customs']

И посмотрим на сам дата-сет

In [4]:
df_test.sample(10)

,body_type,brand,car_url,color,complectation,description,engine_vol,engine_power,equipment,fuel_type,image,mileage,model_date,model_info,model_name,name,doors_num,parsing_unixtime,price_currency,production_date,sell_id,super_gen,configuration,transmission,vendor,num_owners,ownership,docs,drive,steering,state,customs
8216,седан,BMW,https://auto.ru/cars/used/sale/bmw/5er/1083370...,чёрный,"{""id"":""21054013"",""name"":""M550d xDrive"",""availa...","Обслуживался в БорисХоф (заказы на руках), тех...",3.0 LTR,381 N12,"{""cruise-control"":true,""asr"":true,""tinted-glas...",дизель,https://autoru.naydex.net/lICN99c69/9bc7d3Rr7k...,75000,2013,"{""code"":""5ER"",""name"":""5 серии"",""ru_name"":""5 се...",5ER,M550d xDrive 3.0d AT (381 л.с.) 4WD,4,1603110285,RUB,2015,1083370160,"{""id"":""10436696"",""name"":""M550d xDrive"",""namepl...",SEDAN AUTOMATIC 3.0,автоматическая,EUROPEAN,3 или более,5 лет и 1 месяц,Оригинал,полный,Левый,Не требует ремонта,Растаможен
34394,седан,NISSAN,https://auto.ru/cars/used/sale/nissan/almera_c...,серый,NaN,"Собственник. Машина в хорошем состоянии, есть ...",1.6 LTR,107 N12,"{""engine-proof"":true,""asr"":true,""airbag-driver...",бензин,https://autoru.naydex.net/16IvE9y79/a7f8demo/N...,136000,2006,"{""code"":""ALMERA_CLASSIC"",""name"":""Almera Classi...",ALMERA_CLASSIC,1.6 AT (107 л.с.),4,1603683089,RUB,2009,1101370035,"{""id"":""2404985"",""displacement"":1596,""engine_ty...",SEDAN AUTOMATIC 1.6,автоматическая,JAPANESE,3 или более,2 года и 4 месяца,Оригинал,передний,Левый,Не требует ремонта,Растаможен
24585,минивэн,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/cara...,чёрный,"{""id"":""20959322"",""name"":""Comfortline"",""availab...","Бережная эксплуатация,на холодную не запускалс...",2.0 LTR,102 N12,"{""esp"":true,""airbag-driver"":true,""aux"":true,""i...",дизель,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,54500,2015,"{""code"":""CARAVELLE"",""name"":""Caravelle"",""ru_nam...",CARAVELLE,2.0d MT (102 л.с.),4,1603218146,RUB,2017,1101146798,"{""id"":""20621090"",""displacement"":1968,""engine_t...",MINIVAN MECHANICAL 2.0,механическая,EUROPEAN,1 владелец,2 года и 11 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен
10071,седан,BMW,https://auto.ru/cars/used/sale/bmw/7er/1099912...,чёрный,"{""id"":""21038606"",""name"":""740Li xDrive"",""availa...","В отличном состоянии, обслуживался у оф дилера...",3.0 LTR,326 N12,"{""cruise-control"":true,""asr"":true,""roller-blin...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,91000,2015,"{""code"":""7ER"",""name"":""7 серии"",""ru_name"":""7 се...",7ER,740Li xDrive 3.0 AT (326 л.с.) 4WD,4,1603117652,RUB,2017,1099912984,"{""id"":""20765229"",""name"":""740L"",""nameplate"":""74...",SEDAN AUTOMATIC 3.0,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
13946,седан,NISSAN,https://auto.ru/cars/used/sale/nissan/teana/10...,чёрный,NaN,"Авто в хорошем состоянии, сделано большое ТО в...",2.5 LTR,167 N12,NaN,бензин,https://autoru.naydex.net/G87gDZg56/28c7a8vqVE...,220000,2011,"{""code"":""TEANA"",""name"":""Teana"",""ru_name"":""Тиан...",TEANA,2.5 CVT (167 л.с.) 4WD,4,1603280365,RUB,2011,1099950014,"{""id"":""20194520"",""displacement"":2488,""engine_t...",SEDAN VARIATOR 2.5,вариатор,JAPANESE,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
16547,внедорожник 5 дв.,MERCEDES,https://auto.ru/cars/used/sale/mercedes/m_klas...,чёрный,NaN,"Мерседес Мл 63 ( ml63 amg) , юридически чистая...",6.2 LTR,510 N12,NaN,бензин,https://autoru.naydex.net/ESooa9149/468b9eoMem...,123000,2008,"{""code"":""M_KLASSE_AMG"",""name"":""M-Класс AMG"",""r...",M_KLASSE_AMG,63 AMG 6.2 AT (510 л.с.) 4WD,5,1603244368,RUB,2010,1096753480,"{""id"":""2307891"",""name"":""63 AMG"",""nameplate"":""6...",ALLROAD_5_DOORS AUTOMATIC 6.2,автоматическая,EUROPEAN,3 или более,NaN,Дубликат,полный,Левый,Не требует ремонта,Растаможен
33860,седан,AUDI,https://auto.ru/cars/used/sale/audi/a6/1101367...,чёрный,NaN,"Авто в хорошем сост

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   body_type         34686 non-null  object
 1   brand             34686 non-null  object
 2   car_url           34686 non-null  object
 3   color             34686 non-null  object
 4   complectation     6418 non-null   object
 5   description       34686 non-null  object
 6   engine_vol        34686 non-null  object
 7   engine_power      34686 non-null  object
 8   equipment         24690 non-null  object
 9   fuel_type         34686 non-null  object
 10  image             34686 non-null  object
 11  mileage           34686 non-null  int64 
 12  model_date        34686 non-null  int64 
 13  model_info        34686 non-null  object
 14  model_name        34686 non-null  object
 15  name              34686 non-null  object
 16  doors_num         34686 non-null  int64 
 17  parsing_unix

В некоторых столбцах есть пропуски (о них чуть позже), много строк, мало цифр - придется попотеть(

### 1.1 Очистка от мусора

Посмотрим на уникальные значения в каждом столбце:

In [6]:
df_test.nunique().sort_values()

customs                 1
price_currency          1
state                   1
steering                2
docs                    2
vendor                  2
drive                   3
num_owners              3
transmission            4
fuel_type               5
doors_num               5
brand                  12
color                  16
body_type              24
engine_vol             55
model_date             66
production_date        69
ownership             275
engine_power          315
model_name            544
configuration         634
model_info            954
complectation        2364
name                 2780
super_gen            5890
mileage             11268
equipment           23705
description         31732
image               34557
parsing_unixtime    34686
car_url             34686
sell_id             34686
dtype: int64

3 столбца содержат всего по 1 уникальному значению, т.е. их наличие никак не влияет на модель. Взглянем на них:

In [7]:
df_test.head(1)[['customs', 'price_currency', 'state']]

,customs,price_currency,state
0,Растаможен,RUB,Не требует ремонта


Искать и парсить будем только объявления с растаможенными, не требующими ремонта автомобилями. цены будем получать в рублях.
Эти столбцы можно удалить. Также сразу удалим отсальные столбцы, не несущие полезной для модели информации.

In [8]:
df_test.drop(columns=['customs', 'price_currency', 'state', 'sell_id', 'car_url', 'image'], inplace=True)

### 1.2 Бинарные столбцы
Теперь разберемся со столбцами, в которых по 2-3 уникальных значения (т.е. по сути с категоральными столбцами)

In [9]:
df_test.steering.unique()

array(['Левый', 'Правый'], dtype=object)

In [10]:
# превратим категоральный признак в бинарный, проставив 1 везде, где руль справа
df_test.steering = (df_test.steering == 'Правый').astype('int')

In [11]:
df_test.docs.unique()

array(['Оригинал', 'Дубликат', nan], dtype=object)

In [12]:
# найдем наиболее частое значение
docs_fill = df_test.docs.value_counts().index[0] 
# заполним им все пропуски
# а затем превратим категоральный признак в бинарный, проставив 1 везде, где ПТС в виде дубликата
df_test.docs = (df_test.docs.fillna(docs_fill) == 'Дубликат').astype('int')

In [13]:
df_test.vendor.unique()

array(['EUROPEAN', 'JAPANESE'], dtype=object)

In [14]:
# затем превратим категоральный признак в бинарный, проставив 1 везде, где производитель из Японии
df_test.vendor = (df_test.vendor == 'JAPANESE').astype('int')

### 1.3 Категоральные столбцы

In [15]:
df_test.drive.unique()

array(['передний', 'полный', 'задний'], dtype=object)

Со столбцом `drive` пока ничего не будем делать

In [16]:
df_test.num_owners.unique()

array(['3 или более', '1\xa0владелец', '2\xa0владельца'], dtype=object)

По сути это числовой столбец, из которого достаточно взять исключительно первый символ

In [17]:
df_test.num_owners = df_test.num_owners.str[0].astype('int')

### 1.4 Составные столбцы

Для анализа составных столбцов добавим вспомогательные функции

In [18]:
def get_word(column, index=-1):
    return column.apply(lambda s: s.split()[index]) if index != -1 else column

def print_unique(column, index=-1):
    col_unique = get_word(column, index).unique()
    print(len(col_unique))
    print(col_unique)

Теперь разберем столбец `configuration`

In [19]:
print_unique(df_test.configuration, 0)

24
['LIFTBACK' 'ALLROAD_5_DOORS' 'HATCHBACK_5_DOORS' 'SEDAN' 'COMPACTVAN'
 'WAGON_5_DOORS' 'PICKUP_ONE' 'HATCHBACK_3_DOORS' 'COUPE' 'CABRIO'
 'MINIVAN' 'PICKUP_TWO' 'ALLROAD_3_DOORS' 'ROADSTER' 'MICROVAN'
 'SEDAN_2_DOORS' 'COUPE_HARDTOP' 'FASTBACK' 'TARGA' 'ALLROAD_OPEN'
 'LIMOUSINE' 'PICKUP_ONE_HALF' 'SEDAN_HARDTOP' 'VAN']


In [20]:
print_unique(df_test.body_type)

24
['лифтбек' 'внедорожник 5 дв.' 'хэтчбек 5 дв.' 'седан' 'компактвэн'
 'универсал 5 дв.' 'пикап одинарная кабина' 'хэтчбек 3 дв.' 'купе'
 'кабриолет' 'минивэн' 'пикап двойная кабина' 'внедорожник 3 дв.'
 'родстер' 'микровэн' 'седан 2 дв.' 'купе-хардтоп' 'фастбек' 'тарга'
 'внедорожник открытый' 'лимузин' 'пикап полуторная кабина'
 'седан-хардтоп' 'фургон']


In [21]:
print_unique(df_test.configuration, 1)

4
['ROBOT' 'MECHANICAL' 'AUTOMATIC' 'VARIATOR']


In [22]:
print_unique(df_test.transmission)

4
['роботизированная' 'механическая' 'автоматическая' 'вариатор']


In [23]:
print_unique(df_test.engine_vol, 0)

55
['1.2' '1.6' '1.8' '2.0' '1.4' '1.3' '1.0' '3.6' '1.5' '1.9' '2.8' '1.1'
 '2.5' '4.2' '3.0' '4.0' '5.9' '2.7' '3.1' '2.4' '5.2' '3.2' '4.1' '6.3'
 '2.3' '6.0' '2.2' '3.7' '2.9' '5.0' '3.3' '2.1' '2.6' 'LTR' '3.5' '1.7'
 '0.7' '4.4' '4.8' '5.4' '6.6' '4.9' '3.8' '3.4' '3.9' '4.6' '5.6' '4.5'
 '5.5' '6.2' '4.7' '4.3' '5.8' '5.3' '5.7']


Первое слово внутри столбца полностью соответствует столбцу `body_type`, а второе - столбцу `transmission`, а третье - вроде как соответствует столбцу `engine_vol`, который, в свою очередь сам составной. такое дублирование не нужно, исправим

In [24]:
# в столбцах body_type и transmission сохраним более привлекательные латинские значения
df_test.body_type = get_word(df_test.configuration, 0)
df_test.transmission = get_word(df_test.configuration, 1)
# в столбца engine_vol избавимся от 'LTR' и превратим значение в число
df_test.engine_vol = df_test.engine_vol.apply(lambda s: '0.0' if s == ' LTR' else s.split()[0]).astype('float')
df_test.drop(columns='configuration', inplace=True)

In [25]:
df_test.head()

,body_type,brand,color,complectation,description,engine_vol,engine_power,equipment,fuel_type,mileage,model_date,model_info,model_name,name,doors_num,parsing_unixtime,production_date,super_gen,transmission,vendor,num_owners,ownership,docs,drive,steering
0,LIFTBACK,SKODA,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,74000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.2 AMT (105 л.с.),5,1603226273,2014,"{""id"":""10373605"",""displacement"":1197,""engine_t...",ROBOT,0,3,NaN,0,передний,0
1,LIFTBACK,SKODA,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,60563,2017,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 MT (110 л.с.),5,1603226277,2017,"{""id"":""20913311"",""displacement"":1598,""engine_t...",MECHANICAL,0,1,NaN,0,передний,0
2,LIFTBACK,SKODA,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,88000,2013,"{""code"":""SUPERB"",""name"":""Superb"",""ru_name"":""Су...",SUPERB,DSG 1.8 AMT (152 л.с.),5,1603226280,2014,"{""id"":""20026323"",""nameplate"":""DSG"",""displaceme...",ROBOT,0,1,NaN,0,передний,0
3,LIFTBACK,SKODA,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,95000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 AT (110 л.с.),5,1603226284,2014,"{""id"":""20105521"",""displacement"":1598,""engine_t...",AUTOMATIC,0,1,NaN,0,передний,0
4,LIFTBACK,SKODA,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,58536,2008,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.8 AT (152 л.с.),5,1603226288,2012,"{""id"":""4561004"",""displacement"":1798,""engine_ty...",AUTOMATIC,0,1,NaN,0,передний,0


Теперь столбец `engine_power`. Похоже, что в нем второе "слово" не имеет смысловой нагрузки для модели

In [26]:
print_unique(df_test.engine_power, 1)

1
['N12']


Так и есть. А первое "слово" также присутствует в столбце `name`, который в свою очередь, также дублирует столбцы `engine_vol` и `transmission`. Исправляем:

In [27]:
df_test.engine_power = get_word(df_test.engine_power, 0).astype('int')
df_test.drop(columns='name', inplace=True)

In [28]:
df_test.sample()

,body_type,brand,color,complectation,description,engine_vol,engine_power,equipment,fuel_type,mileage,model_date,model_info,model_name,doors_num,parsing_unixtime,production_date,super_gen,transmission,vendor,num_owners,ownership,docs,drive,steering
12292,SEDAN,NISSAN,чёрный,NaN,Автомат!! Не вариатор! Ввезена из Британии в 2...,2.4,173,NaN,бензин,165000,2005,"{""code"":""TEANA"",""name"":""Teana"",""ru_name"":""Тиан...",TEANA,4,1603274107,2007,"{""id"":""3480707"",""displacement"":2349,""engine_ty...",AUTOMATIC,1,3,2 года и 5 месяцев,0,передний,0


Столбец `parsing_time` содержит дату/время выгрузки объявления. Для обучения модели он вряд ли значим, а вот для сабмишна - да, он необходим, чтобы сделать поправку хотя бы на валютный курс. Сохраним его копию, а из дата-сета удалим

In [29]:
parsing_time = df_test.parsing_unixtime.copy()
df_test.drop(columns='parsing_unixtime', inplace=True)

In [30]:
parsing_time

0        1603226273
1        1603226277
2        1603226280
3        1603226284
4        1603226288
            ...    
34681    1603707643
34682    1603707651
34683    1603707843
34684    1603707909
34685    1603710264
Name: parsing_unixtime, Length: 34686, dtype: int64